# 🏥 Medical Image Segmentation - Results & Evaluation

This notebook evaluates all trained models and creates a comprehensive results summary.

**Prerequisites:** You must have already run:
- `COLAB_TRAIN_UNET.ipynb`
- `COLAB_TRAIN_UNETPP.ipynb`
- `COLAB_TRAIN_TRANSUNET.ipynb`

---

## Step 1: Setup Environment

In [ ]:
# Verify GPU is available
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")

## Step 2: Clone Repository & Install Dependencies

In [ ]:
# Clone the repository
!git clone https://github.com/Prabhat9801/Medical-Image-Segmentation.git
%cd Medical-Image-Segmentation

# Install required packages
!pip install -q timm albumentations opencv-python-headless tqdm

## Step 3: Mount Google Drive & Load Results

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import shutil

# Copy all experiment results from Google Drive
results_dir = "/content/drive/MyDrive/medical_segmentation_results"
local_exp_dir = "/content/Medical-Image-Segmentation/experiments"

# Create experiments directory
os.makedirs(local_exp_dir, exist_ok=True)

# Copy UNet results
if os.path.exists(f"{results_dir}/unet_experiments"):
    !cp -r {results_dir}/unet_experiments/* {local_exp_dir}/
    print("✅ UNet results loaded")

# Copy UNet++ results
if os.path.exists(f"{results_dir}/unetpp_experiments"):
    !cp -r {results_dir}/unetpp_experiments/* {local_exp_dir}/
    print("✅ UNet++ results loaded")

# Copy TransUNet results
if os.path.exists(f"{results_dir}/transunet_experiments"):
    !cp -r {results_dir}/transunet_experiments/* {local_exp_dir}/
    print("✅ TransUNet results loaded")

# List all experiments
print("\n📁 Available experiments:")
!ls -la {local_exp_dir}

## Step 4: Extract Data (for evaluation)

In [ ]:
import zipfile
import pandas as pd

# Paths
zip_path = "/content/drive/MyDrive/isic_processed_256.zip"
final_path = "/content/Medical-Image-Segmentation/data/processed/isic"

print("📦 Extracting data...")

# Remove existing if present
if os.path.exists(final_path):
    shutil.rmtree(final_path)

# Create the directory structure
os.makedirs(final_path, exist_ok=True)
os.makedirs(os.path.join(final_path, "images"), exist_ok=True)
os.makedirs(os.path.join(final_path, "masks"), exist_ok=True)

# Extract and fix Windows paths
with zipfile.ZipFile(zip_path, 'r') as z:
    for file_info in z.filelist:
        orig_name = file_info.filename
        fixed_name = orig_name.replace('\\', '/')
        
        if fixed_name.startswith('isic/'):
            fixed_name = fixed_name[5:]
        
        if 'images/' in fixed_name or (fixed_name.endswith('.png') and 'mask' not in fixed_name.lower()):
            filename = os.path.basename(fixed_name)
            target_path = os.path.join(final_path, "images", filename)
        elif 'masks/' in fixed_name or '_mask.png' in fixed_name:
            filename = os.path.basename(fixed_name)
            target_path = os.path.join(final_path, "masks", filename)
        elif fixed_name.endswith('splits.csv'):
            target_path = os.path.join(final_path, "splits.csv")
        else:
            continue
        
        with z.open(file_info) as source, open(target_path, 'wb') as target:
            shutil.copyfileobj(source, target)

# Fix splits.csv paths
splits_path = os.path.join(final_path, "splits.csv")
if os.path.exists(splits_path):
    df = pd.read_csv(splits_path)
    df['image_path'] = df['image_path'].apply(
        lambda x: os.path.join(final_path, "images", os.path.basename(x))
    )
    df['mask_path'] = df['mask_path'].apply(
        lambda x: os.path.join(final_path, "masks", os.path.basename(x))
    )
    df.to_csv(splits_path, index=False)

print("✅ Data extraction complete!")

## Step 5: Evaluate All Models

In [ ]:
import glob

# Find all experiment directories
exp_dirs = glob.glob(f"{local_exp_dir}/*")
exp_dirs = [d for d in exp_dirs if os.path.isdir(d)]

print(f"Found {len(exp_dirs)} experiments to evaluate\n")

# Evaluate each experiment
for exp_dir in sorted(exp_dirs):
    exp_name = os.path.basename(exp_dir)
    print(f"\n{'='*60}")
    print(f"Evaluating: {exp_name}")
    print(f"{'='*60}")
    
    # Run evaluation
    !python -m src.eval --exp_dir {exp_dir}
    
    print(f"\n✅ {exp_name} evaluation complete!")

## Step 6: Generate Results Summary

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Collect all results
results = []

for exp_dir in sorted(exp_dirs):
    metrics_file = os.path.join(exp_dir, "test_metrics.json")
    if os.path.exists(metrics_file):
        with open(metrics_file, 'r') as f:
            metrics = json.load(f)
        
        # Extract model and data fraction from directory name
        exp_name = os.path.basename(exp_dir)
        parts = exp_name.split('_')
        
        model = parts[0]
        data_frac = parts[1] if len(parts) > 1 else "unknown"
        
        results.append({
            'Model': model,
            'Data Fraction': data_frac,
            'Dice Score': metrics.get('dice', 0),
            'IoU': metrics.get('iou', 0),
            'Pixel Accuracy': metrics.get('pixel_accuracy', 0)
        })

# Create DataFrame
df_results = pd.DataFrame(results)

# Display results
print("\n" + "="*80)
print("📊 RESULTS SUMMARY")
print("="*80)
print(df_results.to_string(index=False))

# Save to CSV
results_csv = "/content/Medical-Image-Segmentation/results_summary.csv"
df_results.to_csv(results_csv, index=False)
print(f"\n✅ Results saved to: {results_csv}")

## Step 7: Create Comparison Plots

In [ ]:
# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (15, 10)

# Create subplots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Dice Score by Model and Data Fraction
pivot_dice = df_results.pivot(index='Data Fraction', columns='Model', values='Dice Score')
pivot_dice.plot(kind='bar', ax=axes[0, 0], rot=0)
axes[0, 0].set_title('Dice Score Comparison', fontsize=14, fontweight='bold')
axes[0, 0].set_ylabel('Dice Score')
axes[0, 0].legend(title='Model')
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: IoU by Model and Data Fraction
pivot_iou = df_results.pivot(index='Data Fraction', columns='Model', values='IoU')
pivot_iou.plot(kind='bar', ax=axes[0, 1], rot=0)
axes[0, 1].set_title('IoU Score Comparison', fontsize=14, fontweight='bold')
axes[0, 1].set_ylabel('IoU Score')
axes[0, 1].legend(title='Model')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Pixel Accuracy by Model and Data Fraction
pivot_acc = df_results.pivot(index='Data Fraction', columns='Model', values='Pixel Accuracy')
pivot_acc.plot(kind='bar', ax=axes[1, 0], rot=0)
axes[1, 0].set_title('Pixel Accuracy Comparison', fontsize=14, fontweight='bold')
axes[1, 0].set_ylabel('Pixel Accuracy')
axes[1, 0].legend(title='Model')
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Overall Performance Heatmap
heatmap_data = df_results.pivot_table(
    index='Model', 
    columns='Data Fraction', 
    values='Dice Score'
)
sns.heatmap(heatmap_data, annot=True, fmt='.3f', cmap='YlGnBu', ax=axes[1, 1])
axes[1, 1].set_title('Dice Score Heatmap', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('/content/Medical-Image-Segmentation/results_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Comparison plots created!")

## Step 8: Find Best Models

In [ ]:
# Find best model overall
best_overall = df_results.loc[df_results['Dice Score'].idxmax()]

print("\n" + "="*80)
print("🏆 BEST MODEL OVERALL")
print("="*80)
print(f"Model: {best_overall['Model']}")
print(f"Data Fraction: {best_overall['Data Fraction']}")
print(f"Dice Score: {best_overall['Dice Score']:.4f}")
print(f"IoU: {best_overall['IoU']:.4f}")
print(f"Pixel Accuracy: {best_overall['Pixel Accuracy']:.4f}")

# Find best model for each data fraction
print("\n" + "="*80)
print("🏆 BEST MODEL PER DATA FRACTION")
print("="*80)

for frac in df_results['Data Fraction'].unique():
    frac_data = df_results[df_results['Data Fraction'] == frac]
    best_frac = frac_data.loc[frac_data['Dice Score'].idxmax()]
    print(f"\n{frac}:")
    print(f"  Model: {best_frac['Model']}")
    print(f"  Dice Score: {best_frac['Dice Score']:.4f}")

## Step 9: Save All Results to Google Drive

In [ ]:
# Create final results directory
final_results_dir = "/content/drive/MyDrive/medical_segmentation_results/final_results"
os.makedirs(final_results_dir, exist_ok=True)

# Copy results summary
!cp /content/Medical-Image-Segmentation/results_summary.csv {final_results_dir}/
!cp /content/Medical-Image-Segmentation/results_comparison.png {final_results_dir}/

# Copy all experiment results
!cp -r /content/Medical-Image-Segmentation/experiments {final_results_dir}/

print("\n✅ All results saved to Google Drive!")
print(f"📁 Location: {final_results_dir}")

## 🎉 Evaluation Complete!

**Results saved to:**
- CSV: `results_summary.csv`
- Plots: `results_comparison.png`
- Google Drive: `/content/drive/MyDrive/medical_segmentation_results/final_results/`

**Next steps:**
1. Download results from Google Drive
2. Update your local `reports/report.md`
3. Create final presentation/documentation